<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/Sojeong_Cho/Naive_Bayes_Classifier(NBC)_%EA%B5%AC%ED%98%84_%EC%A1%B0%EC%86%8C%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
import pandas as pd
import numpy as np
import math

In [132]:
# 1 = spam, 0 = normal

mails =  ['me free lottery', 'free get free you', 'you free scholarship', 'free to contact me', 'you won award', 'you ticket lottery']
classification = [1, 1, 0, 0, 0, 1]

# 토크나이징
mails = [mail.split() for mail in mails]
mails

mails_1 = []

for i in range(len(mails)):
    mails_1.append(' '.join(mails[i]))
mails_1

text = np.array([mails_1, classification])

mail_frame = pd.DataFrame(text.T,columns=['words', 'classification'])
mail_frame

,words,classification
0,me free lottery,1
1,free get free you,1
2,you free scholarship,0
3,free to contact me,0
4,you won award,0
5,you ticket lottery,1


In [133]:
# 토큰 수 세기

mails_token = []
for b in mails:
    for token in b:
        mails_token.append(token)

mails_token = np.unique(mails_token)

print(mails_token)

['award' 'contact' 'free' 'get' 'lottery' 'me' 'scholarship' 'ticket' 'to'
 'won' 'you']


In [134]:
# spam and normal의 각각 토큰 개수 구하기

spam_list = []
normal_list = []

for i in range(len(mails_token)):
    spam_cnt = 0
    normal_cnt = 0

    for j in range(len(mails)):
        if mails_token[i] in mails[j]:
            if classification[j] == 1:
                spam_cnt+=mails[j].count(mails_token[i])
            elif classification[j] == 0:
                normal_cnt+=mails[j].count(mails_token[i])
            else:
                pass
                
    spam_list.append(spam_cnt)
    normal_list.append(normal_cnt)

print("spam :", spam_list)
print("normal :", normal_list)

spam : [0, 0, 3, 1, 2, 1, 0, 1, 0, 0, 2]
normal : [1, 1, 2, 0, 0, 1, 1, 0, 1, 1, 2]


In [135]:
# 사전확률 구하기

p_spam = []
p_normal = []

for i in range(len(mails_token)):
    prob_spam = (0.5+float(str(spam_list[i])))/(2*0.5+sum(spam_list))*100
    prob_normal = (0.5+float(str(normal_list[i])))/(2*0.5+sum(normal_list))*100
    
    p_spam.append("%0.2f" %prob_spam)
    p_normal.append("%0.2f" %prob_normal)

print("spam 사전확률 :", p_spam)
print("normal 사전확률 :", p_normal)

spam 사전확률 : ['4.55', '4.55', '31.82', '13.64', '22.73', '13.64', '4.55', '13.64', '4.55', '4.55', '22.73']
normal 사전확률 : ['13.64', '13.64', '22.73', '4.55', '4.55', '13.64', '13.64', '4.55', '13.64', '13.64', '22.73']


In [136]:
# log

log_p_spam = []
log_p_normal = []

for i in range(len(mails_token)):
     log_prob_spam = np.log((0.5+float(str(spam_list[i])))/(2*0.5+sum(spam_list)))
     log_prob_normal = np.log((0.5+float(str(normal_list[i])))/(2*0.5+sum(normal_list)))
     
     log_p_spam.append("%0.4f"%log_prob_spam)
     log_p_normal.append("%0.4f"%log_prob_normal)
    
print("log spam :", log_p_spam)
print("log normal :", log_p_normal)

log spam : ['-3.0910', '-3.0910', '-1.1451', '-1.9924', '-1.4816', '-1.9924', '-3.0910', '-1.9924', '-3.0910', '-3.0910', '-1.4816']
log normal : ['-1.9924', '-1.9924', '-1.4816', '-3.0910', '-3.0910', '-1.9924', '-1.9924', '-3.0910', '-1.9924', '-1.9924', '-1.4816']


In [137]:
# 시각화

table = np.array([mails_token,spam_list,normal_list,p_spam,p_normal,log_p_spam,log_p_normal])
pd.DataFrame(table.T, columns=['words','spam','normal','P(spam)','P(normal)','Log(P(spam))','Log(P(normal))'])

,words,spam,normal,P(spam),P(normal),Log(P(spam)),Log(P(normal))
0,award,0,1,4.55,13.64,-3.0910,-1.9924
1,contact,0,1,4.55,13.64,-3.0910,-1.9924
2,free,3,2,31.82,22.73,-1.1451,-1.4816
3,get,1,0,13.64,4.55,-1.9924,-3.0910
4,lottery,2,0,22.73,4.55,-1.4816,-3.0910
5,me,1,1,13.64,13.64,-1.9924,-1.9924
6,scholarship,0,1,4.55,13.64,-3.0910,-1.9924
7,ticket,1,0,13.64,4.55,-1.9924,-3.0910
8,to,0,1,4.55,13.64,-3.0910,-1.9924
9,won,0,1,4.55,13.64,-3.0910,-1.9924


In [138]:
spam_prob = np.log(classification.count(1)/len(classification))
normal_prob = np.log(classification.count(0)/len(classification))

#'free'와 'lottery'에 대한 스팸 메일 확률 구하기

a_1 = math.exp(float(log_p_normal[2])+(float(log_p_normal[4]))+(float("%0.4f"%normal_prob)))
a_2 = math.exp(float(log_p_spam[2])+(float(log_p_spam[4]))+(float("%0.4f"%spam_prob)))

print('스팸일 확률: ', (a_2/(a_1+a_2))*100, '정상 메일일 확률: ', (a_1/(a_1+a_2))*100)

스팸일 확률:  87.49988899428504 정상 메일일 확률:  12.500111005714952
